<a href="https://colab.research.google.com/github/Piyu569/SparkProject01/blob/dev/SparkProject01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3b0e3cb5d4cd657c53b27eac3094b91c7fcfb0d75414289545b9c29adf5b64d9
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName("banking_Pro").getOrCreate()

In [ ]:
#@title Credit-Card dataSet

In [9]:
df = spark.read.csv("/credit card.csv",inferSchema = True,header =True)

In [35]:
df.printSchema()

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



In [12]:
df.count()

10000

# This is the Distinct count of the members who are eligible for credit card

In [23]:
from pyspark.sql.functions import col
eligible = df.filter(df["CreditScore"] > 700).distinct().count()
print(eligible)

3116


# List of the Distinct Members

In [41]:
from pyspark.sql.functions import avg
list = df.select("CreditScore","Age","NumOfProducts").filter(df["CreditScore"]>700)
list.show()

+-----------+---+-------------+
|CreditScore|Age|NumOfProducts|
+-----------+---+-------------+
|        850| 43|            1|
|        822| 50|            2|
|        726| 24|            2|
|        732| 41|            2|
|        846| 38|            1|
|        756| 36|            1|
|        722| 29|            2|
|        804| 33|            1|
|        850| 36|            1|
|        834| 49|            1|
|        776| 32|            2|
|        829| 27|            1|
|        776| 37|            2|
|        788| 33|            2|
|        725| 19|            1|
|        742| 35|            1|
|        751| 36|            2|
|        735| 43|            2|
|        738| 58|            4|
|        813| 29|            1|
+-----------+---+-------------+
only showing top 20 rows



In [42]:
df.show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888| Mitchell|        850|    Spain|F

# List of members active and eligible for credit card

In [52]:
activeandeligible = df.filter((df["CreditScore"]>700).isNotNull() & (df["IsActiveMember"] == 1).isNotNull()).count()
print(activeandeligible)


10000


# Loan DataSet

In [53]:
df2 = spark.read.csv("/loan.csv",inferSchema = True,header = "True")

In [55]:
df2.show()
df2.printSchema()

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|           HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|          PROFESSOR|       MARRIED|          6| 51000|      19999|            4|          SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|           

# Number of loans in each category

In [57]:
numberofloan = df2.groupBy("Loan Category").count().show()

+------------------+-----+
|     Loan Category|count|
+------------------+-----+
|           HOUSING|   67|
|        TRAVELLING|   53|
|       BOOK STORES|    7|
|       AGRICULTURE|   12|
|         GOLD LOAN|   77|
|  EDUCATIONAL LOAN|   20|
|        AUTOMOBILE|   60|
|          BUSINESS|   24|
|COMPUTER SOFTWARES|   35|
|           DINNING|   14|
|          SHOPPING|   35|
|       RESTAURANTS|   41|
|       ELECTRONICS|   14|
|          BUILDING|    7|
|        RESTAURANT|   20|
|   HOME APPLIANCES|   14|
+------------------+-----+



# Number of people who have taken more than 1 lack loan

In [70]:
loanmorethanonelakh = df2.filter(df2["Loan Amount"]>"1,00,000").count()
print(loanmorethanonelakh)

379


# Number of people with income greater than 60000 rupees

In [72]:
incomemorethansistythousand = df2.filter(df2["Income"]>60000).count() 
print(incomemorethansistythousand)

198


#Number of people with 2 or more returned cheques and income less than 50000


In [78]:
df2.filter((df2[" Returned Cheque"]>"1") & (df2["Income"]<"50000")).count()


137